In [ ]:
import os

try:
    if CONFIGURED_DIRS:  # type: ignore
        pass
except:
    PLAYGROUND_DIR = os.getcwd()
    os.chdir("../src")

CONFIGURED_DIRS = True

print(f"Playground directory: {PLAYGROUND_DIR}")  # type: ignore
print(f"Working directory: {os.getcwd()}")

In [ ]:
from IPython.display import Audio

from common.roll import Roll, RollExportConfig
from common.note_collection import NoteCollection
from common.structures.note import Note
from common.structures.chord import Chord, ChordQuality
from common.structures.pitch import Pitch
from common.structures.interval import Interval

from generation.chord_progression import ChordProgression
from generation.instruments.voice import Voice
from generation.instruments.piano import Piano

In [ ]:
key = Pitch.from_str("C")

I = Chord(key, ChordQuality.Maj)
ii = Chord(key + Interval.from_str("2"), ChordQuality.Min)
IV = Chord(key + Interval.from_str("4"), ChordQuality.Maj)
V = Chord(key + Interval.from_str("5"), ChordQuality.Maj)

roll = Roll(
    beats_per_minute=110,
    time_signature=(4, 4),
)
voice = roll.add_instrument("Voice", Voice)
assert type(voice) is Voice
piano = roll.add_instrument("Stride Piano", Piano)
assert type(piano) is Piano

note_collection = NoteCollection()
note_collection.add(
    Note(Pitch.from_str("C5"), roll.Time(0, 0), roll.Duration(2)),
    Note(Pitch.from_str("C#5"), roll.Time(0, 3), roll.Duration(1)),
    Note(Pitch.from_str("D5"), roll.Time(1, 0), roll.Duration(1)),
    Note(Pitch.from_str("F5"), roll.Time(1, 1), roll.Duration(1)),
    Note(Pitch.from_str("A4"), roll.Time(1, 2), roll.Duration(1)),
    Note(Pitch.from_str("E5"), roll.Time(1, 3), roll.Duration(0.5)),
    Note(Pitch.from_str("C#5"), roll.Time(1, 3.5), roll.Duration(0.5)),
    Note(Pitch.from_str("D5"), roll.Time(2, 0), roll.Duration(2)),
    Note(Pitch.from_str("D#5"), roll.Time(2, 3), roll.Duration(1)),
    Note(Pitch.from_str("E5"), roll.Time(3, 0), roll.Duration(1)),
    Note(Pitch.from_str("G5"), roll.Time(3, 1), roll.Duration(1)),
    Note(Pitch.from_str("G4"), roll.Time(3, 2), roll.Duration(1)),
    Note(Pitch.from_str("D5"), roll.Time(3, 3), roll.Duration(0.5)),
    Note(Pitch.from_str("B4"), roll.Time(3, 3.5), roll.Duration(0.5)),
    Note(Pitch.from_str("C5"), roll.Time(4, 0), roll.Duration(2)),
)

chord_progression = ChordProgression(
    start_time=roll.Time(0, 0),
    end_time=roll.Time(4, 2),
)
chord_progression.add_chords(
    (I, roll.Time(0, 0)),
    (IV, roll.Time(1, 0)),
    (ii, roll.Time(2, 0)),
    (V, roll.Time(3, 0)),
    (I, roll.Time(4, 0)),
    (IV.get_V7(), roll.Time(0, 2)),
    (ii.get_V7(), roll.Time(1, 2)),
    (V.get_V7(), roll.Time(2, 2)),
    (I.get_V7(), roll.Time(3, 2)),
)

final_pitch = I.root.reoctave_near_pitch(Pitch.from_str("C3"))
voice.generate(note_collection, chord_progression)
piano.generate(chord_progression)
piano.notes.add(Note(final_pitch, chord_progression.end_time, roll.Duration(1)))

In [ ]:
OUTPUT_DIR = os.path.join(PLAYGROUND_DIR, "output")
WAV_SRC = os.path.join(OUTPUT_DIR, "output.wav")

roll.export(RollExportConfig(output_path=WAV_SRC))
Audio(WAV_SRC)